# Distributed Parallel Programming Patterns using Open MPI
C adaptation done by Ruth Kurniawati (Westfield State University) using source code from [CSInParallel](https://github.com/csinparallel/CSinParallel.git)

Modified from mpi4py notebook originally written by Libby Shoop, Macalester College

Welcome!

This book contains some examples illustrating the basic fundamental concepts of distributed computing using C code. The type of computing these examples illustrate is called *message passing*. Message passing is a form of programming that is based on processes that communicate with each other to coordinate their work. Message passing can be used on a single multicore computer or with a cluster of computers.

### Software Patterns

Patterns in software are common implementations that have been used over and over by practitioners to accomplish tasks. As practitioners use them repeatedly, the community begins to give them names and catalog them, often turning them into reusable library functions. The examples you will see in this book are based on documented patterns that have been used to solve different problems using message passing between processes. Message passing is one form of distributed computing using processes, which can be used on clusters of computers or multicore machines.

In many of these examples, the pattern's name is part of the C code file's name. You will also see that often the MPI library functions also take on the name of the pattern, and the implementation of those functions themselves contains the pattern that practitioners found themselves using often. These pattern code examples we show you here, dubbed patternlets, are based on original work by Joel Adams:

Adams, Joel C. "Patternlets: A Teaching Tool for Introducing Students to Parallel Design Patterns." 2015 IEEE International Parallel and Distributed Processing Symposium Workshop. IEEE, 2015.

To run these examples, first you may need to install openmpi by running this code. You may need to re-run this if the notebook has been disconnected from the host runtime and has to be restarted. You don't need to run this code if you open this notebook in Google Colab or Binder using one of the links from the README file. 

In [ ]:
!apt install -y openmpi-bin libopenmpi-dev

### New to colab and jupyter notebook?

If you have not used this type of notebook before, these are split into *cells*. The cell you are reading is a text cell, and the cell just above it is also. the cell with [ ] to the left of it is a code cell, which contains C code or code that can be run as if you are in a linux shell. The latter linux shell commands always begin with an exclamation point, !, as the cell above that contains a wget command, used to download the mpi.jar file.

You should execute code cells as you follow along in this notebook. Some are designed for you to re-run after changing them. You can run a cell by hovering over the [ ] and clicking on the arrow symbol.

If you open this notebook in Google Colab, the hamburger icon in the upper left (the one that looks like three __ symbols), toggles the table of contents. Revealing this enables you to navigate to different pattern examples.

The triangle next to some text cells below enables collapsing of sections for faster scrolling.

# Program structure patterns

## Single Program, Multiple Data

This code forms the basis of all of the other examples that follow. It is the fundamental way we structure parallel programs today.


In [1]:
%%writefile spmd.c
/* spmd.c
 * ... illustrates the single program multiple data
 *      (SPMD) pattern using basic MPI commands.
 *
 * Joel Adams, Calvin College, November 2009.
 *
 * Usage: mpirun -np 4 ./spmd
 *
 * Exercise:
 * - Compile and run.
 * - Compare source code to output.
 * - Rerun, using varying numbers of processes
 *    (i.e., vary the argument to 'mpirun -np').
 * - Explain what "multiple data" values this
 *    "single program" is generating.
 */

#include <stdio.h>   // printf()
#include <mpi.h>     // MPI functions

int main(int argc, char** argv) {
    int id = -1, numProcesses = -1, length = -1;
    char myHostName[MPI_MAX_PROCESSOR_NAME];

    MPI_Init(&argc, &argv);
    MPI_Comm_rank(MPI_COMM_WORLD, &id);
    MPI_Comm_size(MPI_COMM_WORLD, &numProcesses);
    MPI_Get_processor_name (myHostName, &length);

    printf("Greetings from process #%d of %d on %s\n",
             id, numProcesses, myHostName);

    MPI_Finalize();
    return 0;
}


Writing spmd.c


Let's examine the variables created in lines 18-29 carefully.

1. *comm* The fundamental notion with this type of computing is a *process* running independently on the computer. With one single program like this, we can specify that we want to start several processes, each of which can **communicate**. The mechanism for communication is initialized when the program starts up, and the object that represents the means of using communication between processes is called MPI.COMM_WORLD.

2. *id* Every process can identify itself with a number. We get that number by asking *comm* for it using Get_rank().

3. *numProcesses* It is helpful to know haw many processes have started up, because this can be specified differently every time you run this type of program. Asking *comm* for it is done with Get_size().

4. *myHostName* When you run this code on a cluster of computers, it is sometimes useful to know which computer is running a certain piece of code. A particular computer is often called a 'host', which is why we call this variable myHostName, and get it by asking *comm* to provide it with Get_processor_name().

These four variables are often used in every MPI program. The first three are often needed for writing correct programs, and the fourth one is often used for debugging and analysis of where certain computations are running.

Next we see how we can compile and use the mpirun program to execute the above C code using 4 processes. The C code will be saved when you execute the previous cell. The next cell will compile and run the saved C source code. We then run the program using mpirun -- the value after -np is the number of processes to use when running the code.

In [ ]:
!mpicc -Wall -o spmd spmd.c
!mpirun --allow-run-as-root -np 4 ./spmd

The fundamental idea of message passing programs can be illustrated like this:

![picture](images/comm_world.png)

Each process is set up within a communication network to be able to communicate with every other process via communication links. Each process is set up to have its own number, or id, which starts at 0.

**Note:** Each process holds its own copies of the above 4 data variables. **So even though there is one single program, it is running multiple times in separate processes, each holding its own data values.** This is the reason for the name of the pattern this code represents: single program, multiple data. The print line at the end of main() represents the multiple different data output produced by each process.


## Master-Worker
This is also a very common pattern used in parallel and distributed programming. Here's the sample small illustrative code. Review it and answer this: What is different between this example and the previous one?


In [2]:
%%writefile masterWorker.c
/* masterWorker.c
 * ... illustrates the basic master-worker pattern in MPI ...
 * Joel Adams, Calvin College, November 2009.
 *
 * Usage: mpirun -np N ./masterWorker
 *
 * Exercise:
 * - Compile and run the program, varying N from 1 through 8.
 * - Explain what stays the same and what changes as the
 *    number of processes changes.
 */

#include <stdio.h>
#include <mpi.h>

int main(int argc, char** argv) {
  int id = -1, numWorkers = -1, length = -1;
  char hostName[MPI_MAX_PROCESSOR_NAME];

  MPI_Init(&argc, &argv);
  MPI_Comm_rank(MPI_COMM_WORLD, &id);
  MPI_Comm_size(MPI_COMM_WORLD, &numWorkers);
  MPI_Get_processor_name (hostName, &length);

  if ( id == 0 ) {  // process 0 is the master 
    printf("Greetings from the master, #%d (%s) of %d processes\n",
             id, hostName, numWorkers);
  } else {          // processes with ids > 0 are workers 
    printf("Greetings from a worker, #%d (%s) of %d processes\n",
             id, hostName, numWorkers);
  }

  MPI_Finalize();
  return 0;
}



Writing masterWorker.c


The answer to the above question illustrates what we can do with this pattern: based on the process id, we can have one process carry out something different than the others. This concept is used a lot as a means to coordinate activities, where one process, often called the master, has the responsibility of handing out work and keeping track of results. We will see this in later examples.

**Note:** By convention, the master coordinating process is usually the process number 0.

In [4]:
!mpicc -Wall -ansi -pedantic -std=c99 -o masterWorker masterWorker.c
!mpirun --allow-run-as-root -np 4 ./masterWorker

Greetings from a worker, #1 (ip-172-31-58-111) of 2 processes
Greetings from the master, #0 (ip-172-31-58-111) of 2 processes


### Exercises:

- Rerun, using varying numbers of processes from 1 through 8 (i.e., vary the argument after -np).
- Explain what stays the same and what changes as the number of processes changes.

# Decomposition using parallel for loop patterns

The most common way to complete a repeated task in any program language is a loop. We use loops because we want to do a certain number of tasks, very often because we want to work on a set of data elements found in a list or an array, or some other data structure. If the work to be done in each loop is independent of previous iterations, we can use separate processes to do parts of the loop independently. This program structure pattern is called the parallel for loop pattern, which is an implementation strategy for decomposition of the work to be done into smaller parts.

## Parallel Loop Split into Equal Sized Chunks

In the code below, notice the use of the variable called `REPS`. This is designed to be the total amount or work, or repetitions, that the for loop is accomplishing. This particular code is designed so that if those repetitions do not divide equally by the number of processes, then the program will stop with a warning message printed by the master process.

Remember that because this is still also a SPMD program, all processes execute the code in the part of the if statement that evaluates to True. Each process has its own id, and we can determine how many processes there are, so we can choose where in the overall number of REPs of the loop each process will execute.

In [ ]:
%%writefile parallelLoopEqualChunks.c
/* parallelLoopEqualChunks.c
 * ... illustrates the parallel for loop pattern in MPI
 *	in which processes perform the loop's iterations in equal-sized 'chunks'
 *	(preferable when loop iterations access memory/cache locations) ...
 * Joel Adams, Calvin College, November 2009.
 *    updated by Libby Shoop, Macalester College, 2017
 *
 * Usage: mpirun -np N ./parallelForEqualChunks
 *
 * Exercise:
 * - Compile and run, varying N: 1, 2, 4, and 8
 * - Change REPS to 16, save, recompile, rerun, varying N again.
 * - Explain how this pattern divides the iterations of the loop
 *    among the processes.
 */

#include <stdio.h> // printf()
#include <mpi.h>   // MPI

int main(int argc, char** argv) {
    const int REPS = 8;                      // repetitions in a loop
    int id = -1, numProcesses = -1;

    MPI_Init(&argc, &argv);
    MPI_Comm_rank(MPI_COMM_WORLD, &id);
    MPI_Comm_size(MPI_COMM_WORLD, &numProcesses);

    // In this example, ensure that the REPS can ben evenly divided by the
    // number of processors and that the number of processes doesn't exceed REPS.
    // If either is the case, stop.
    if ((REPS % numProcesses) == 0 && numProcesses <= REPS) {

      int chunkSize = REPS / numProcesses;      // find chunk size
      int start = id * chunkSize;               // find starting index
      int stop = start + chunkSize;             // find stopping index

      for (int i = start; i < stop; i++) {      // iterate through our range
          printf("Process %d is performing iteration %d\n", id, i);
      }

    } else {
      if (id == 0) {
          printf("Please run with -np divisible by and less than or equal to %d\n.", REPS);
      }
    }

    MPI_Finalize();
    return 0;
}


In [ ]:
!mpicc -Wall -ansi -pedantic -std=c99 -o parallelLoopEqualChunks parallelLoopEqualChunks.c 
!mpirun --allow-run-as-root -np 4 ./parallelLoopEqualChunks

### Exercises

- Run, using these numbers of processes, N: 1, 2, 4, and 8 (i.e., vary the  argument to -np).
- Change REPS to 16 in the code and rerun it. Then rerun with mpirun, varying N again.
- Explain how this pattern divides the iterations of the loop among the processes.

Which of the following is the correct assignment of loop iterations to processes for this code, when REPS is 8 and numProcesses is 4?


![picture](images/decomp_choices.png)

## Parallel for Loop Program Structure: chunks of 1

In the code below, we again use the variable called `REPS` for the total amount or work, or repetitions, that the for loop is accomplishing. This particular code is designed so that the number of repetitions should be more than or equal to the number of processes requested.
.. note:: Typically in real problems, the number of repetitions is much higher than the number of processes. We keep it small here to illustrate what is happening.

Like the last example all processes execute the code in the part of the if statement that evaluates to True. Note that in the for loop in this case we simply have process whose id is 0 start at iteration 0, then skip to 0 + numProcesses for its next iteration, and so on. Similarly, process 1 starts at iteration 1, skipping next to 1+ numProcesses, and continuing until REPs is reached. Each process performs similar single 'slices' or 'chunks of size 1' of the whole loop.


In [ ]:
%%writefile parallelLoopChunksOf1.c
/* parallelLoopChunksOf1.c
 * ... illustrates the parallel for loop pattern in MPI
 *	in which processes perform the loop's iterations in 'chunks'
 *      of size 1 (simple, and useful when loop iterations
 *      do not access memory/cache locations) ...
 * Note this is much simpler than the 'equal chunks' loop.
 * Joel Adams, Calvin College, November 2009.
 *   updated by Libby Shoop, Macalester College, July, 2017
 *
 * Usage: mpirun -np N ./parallelLoopChunksOf1
 *
 * Exercise:
 * - Compile and run, varying N: 1, 2, 3, 4, 5, 6, 7, 8
 * - Change REPS to 16, save, recompile, rerun, varying N again.
 * - Explain how this pattern divides the iterations of the loop
 *    among the processes.
 */

#include <stdio.h>  // printf()
#include <mpi.h>    // MPI

int main(int argc, char** argv) {
    const int REPS = 8;
    int id = -1, numProcesses = -1, i = -1;

    MPI_Init(&argc, &argv);
    MPI_Comm_rank(MPI_COMM_WORLD, &id);
    MPI_Comm_size(MPI_COMM_WORLD, &numProcesses);

    if (numProcesses > REPS) {
      if (id == 0) {
          printf("Please run with -np less than or equal to %d\n.", REPS);
      }
    } else {
      for (i = id; i < REPS; i += numProcesses) {
          printf("Process %d is performing iteration %d\n", id, i);
      }
    }

    MPI_Finalize();
    return 0;
}


In [ ]:
!mpicc -Wall -ansi -pedantic -std=c99 -o parallelLoopChunksOf1 parallelLoopChunksOf1.c 
!mpirun --allow-run-as-root -np 4 ./parallelLoopChunksOf1

### Exercises
- Run, using these numbers of processes, N: 1, 2, 4, and 8
- Compare source code to output.
- Change REPS to 16, save, rerun, varying N again.
- Explain how this pattern divides the iterations of the loop among the processes.

Which of the following is the correct assignment of loop iterations to processes for this code, when REPS is 8 and numProcesses is 4?


![picture](images/decomp_choices.png)

# Point to point communication: the message passing pattern

The fundamental basis of coordination between independent processes is point-to-point communication between processes through the communication links in the MPI.COMM_WORLD. The form of communication is called message passing, where one process **sends** data to another one, who in turn must **receive** it from the sender. This is illustrated as follows:

![picture](images/send_recv.png)

## Message Passing Pattern: Key Problem

The following code represents a common error that many programmers have inadvertently placed in their code. The concept behind this program is that we wish to use communication between pairs of processes, like this:

![picture](images/pair_exchange.png)

For message passing to work between a pair of processes, one must send and the other must receive. If we wish to **exchange** data, then each process will need to perform both a send and a receive.
The idea is that process 0 will send data to process 1, who will receive it from process 0. Process 1 will also send some data to process 0, who will receive it from process 1. Similarly, processes 2 and 3 will exchange messages: process 2 will send data to process 3, who will receive it from process 2. Process 3 will also send some data to process 2, who will receive it from process 3.

If we have more processes, we still want to pair up processes together to exchange messages. The mechanism for doing this is to know your process id. If your id is odd (1, 3 in the above diagram), you will send and receive from your neighbor whose id is id - 1. If your id is even (0, 2), you will send and receive from your neighbor whose id is id + 1. This should work even if we add more than 4 processes, as long as the number of processes is divisible by 2.

![warning sign](images/warning.png)
**Warning** There is a problem with the following code called *deadlock*. This happens when every process is waiting on an action from another process. The program cannot complete. **To stop the program, choose the small square that appears after you choose to run the mpirun cell.**


In [ ]:
%%writefile messagePassingDeadlock.c
/* messagePassingDeadlock.c
 * ... illustrates deadlock with MPI_Send() and MPI_Recv() commands...
 *
 * Joel Adams, Calvin College, November 2009.
 * Modified by Hannah Sonsalla, Macalester College 2017.
 *
 * Usage: mpirun -np N ./messagePassing
 *
 * Exercise:
 * - Compile and run, using more than one process.
 * - Use source code to trace execution.
 * - Why does this fail?
 */

#include <stdio.h>
#include <mpi.h>

int odd(int number) { return number % 2; }

int main(int argc, char** argv) {
    int id = -1, numProcesses = -1;
    int sendValue = -1, receivedValue = -1;
    MPI_Status status;

    MPI_Init(&argc, &argv);
    MPI_Comm_rank(MPI_COMM_WORLD, &id);
    MPI_Comm_size(MPI_COMM_WORLD, &numProcesses);

    if (numProcesses > 1) {
        sendValue = id;
        if ( odd(id) ) {  // odd processors receive from their 'left neighbor', then send
            MPI_Recv(&receivedValue, 1, MPI_INT, id-1, 2,
                       MPI_COMM_WORLD, &status);
            MPI_Send(&sendValue, 1, MPI_INT, id-1, 1, MPI_COMM_WORLD);

        } else {          // even processors receive from their 'right neighbor', then send
            MPI_Recv(&receivedValue, 1, MPI_INT, id+1, 1,
                       MPI_COMM_WORLD, &status);
            MPI_Send(&sendValue, 1, MPI_INT, id+1, 2, MPI_COMM_WORLD);
        }

        printf("Process %d of %d computed %d and received %d\n",
                id, numProcesses, sendValue, receivedValue);
    } else if ( !id) {  // only process 0 does this part
        printf("\nPlease run this program using -np N where N is positive and even.\n\n");
    }

    MPI_Finalize();
    return 0;
}


In [ ]:
!mpicc -Wall -ansi -pedantic -std=c99 -o messagePassingDeadlock messagePassingDeadlock.c 
!mpirun --allow-run-as-root -np 4 ./messagePassingDeadlock

![warning sign](images/warning.png)Remember,**To stop the program, choose the small square that appears after you choose to run the mpirun cell.**

#### What causes the deadlock?

Each process, regardless of its id, will execute a receive request first. In this model, recv is a **blocking** function- it will not continue until it gets data from a send. So every process is blocked waiting to receive a message.

#### Can you think of how to fix this problem?

Since recv is a **blocking** function, we need to have some processes send first, while others correspondingly recv first from those who send first. This provides coordinated exchanges.

Go to the next example to see the solution.


## Message Passing Patterns: avoiding deadlock

Let's look at a few more correct message passing examples.

### Fix the Deadlock

To fix deadlock of the previous example, we coordinate the communication between pairs of processes so that there is an ordering of sends and receives between them.

![Important symbol](images/Important.jpg)**Important:** The new code corrects deadlock with a simple change: odd process sends first, even process receives first. *This is the proper pattern for exchanging data between pairs of processes.*

In [ ]:
%%writefile messagePassing.c
/* messagePassing.c
 * ... illustrates the use of the MPI_Send() and MPI_Recv() commands...
 * Joel Adams, Calvin College, November 2009.
 * Modified by Hannah Sonsalla, Macalester College 2017.
 *
 * Usage: mpirun -np N ./messagePassing
 *
 * Exercise:
 * - Compile and run, using N = 4, 6, 8, and 10 processes.
 * - Use source code to trace execution.
 * - Explain what each process:
 * -- sends
 * -- receives
 * -- outputs.
 * - Run using N = 5 processes. What happens?
 */

#include <stdio.h>
#include <mpi.h>

int odd(int number) { return number % 2; }

int main(int argc, char** argv) {
    int id = -1, numProcesses = -1;
    int sendValue = -1, receivedValue = -1;
    MPI_Status status;

    MPI_Init(&argc, &argv);
    MPI_Comm_rank(MPI_COMM_WORLD, &id);
    MPI_Comm_size(MPI_COMM_WORLD, &numProcesses);

    if (numProcesses > 1) {
        sendValue = id;
        if ( odd(id) ) {  // odd processors send, then receive
            MPI_Send(&sendValue, 1, MPI_INT, id-1, 1, MPI_COMM_WORLD);
            MPI_Recv(&receivedValue, 1, MPI_INT, id-1, 2,
                       MPI_COMM_WORLD, &status);
        } else {          // even processors receive, then send
            MPI_Recv(&receivedValue, 1, MPI_INT, id+1, 1,
                       MPI_COMM_WORLD, &status);
            MPI_Send(&sendValue, 1, MPI_INT, id+1, 2, MPI_COMM_WORLD);
        }

        printf("Process %d of %d computed %d and received %d\n",
                id, numProcesses, sendValue, receivedValue);
    } else if ( !id) {  // only process 0 does this part
        printf("\nPlease run this program using -np N where N is positive and even.\n\n");
    }

    MPI_Finalize();
    return 0;
}


In [ ]:
!mpicc -Wall -ansi -pedantic -std=c99 -o messagePassing messagePassing.c 
!mpirun --allow-run-as-root -np 4 ./messagePassing

### Exercise

- Run, using N = 4, 6, 8, and 10 processes. (Note what happens if you use an odd number instead.)


## Sending data structures
This next example illustrates that we can exchange different arrays of data between processes.


In [ ]:
%%writefile messagePassing2.c
/* messagePassing2.c
 * ... illustrates using MPI_Send() and MPI_Recv() commands on arrays...
 * While this example sends and receives char arrays (strings),
 *  the same approach works on arrays of numbers or other types.
 * Joel Adams, Calvin College, September 2013.
 *
 * Usage: mpirun -np N ./messagePassing2
 *
 * Exercise:
 * - Compile and run, varying N: 1, 2, 4, 8.
 * - Trace execution using source code.
 * - Compare to messagePassing1.c; note send/receive differences.
 */

#include <stdio.h>   // printf()
#include <mpi.h>     // MPI
#include <stdlib.h>  // malloc()
#include <string.h>  // strlen()

int odd(int number) { return number % 2; }

int main(int argc, char** argv) {
    int id = -1, numProcesses = -1, length = -1;
    char * sendString = NULL;
    char * receivedString = NULL;
    char hostName[MPI_MAX_PROCESSOR_NAME];
    MPI_Status status;
    const int SIZE = (32+MPI_MAX_PROCESSOR_NAME) * sizeof(char);

    MPI_Init(&argc, &argv);
    MPI_Comm_rank(MPI_COMM_WORLD, &id);
    MPI_Comm_size(MPI_COMM_WORLD, &numProcesses);
    MPI_Get_processor_name (hostName, &length);

    if (numProcesses > 1 && !odd(numProcesses) ) {
        sendString = (char*) malloc( SIZE );
        receivedString = (char*) malloc( SIZE );
        // sprintf: write to string
        sprintf(sendString, "Process %d is on host \"%s\"", id, hostName);

        if ( odd(id) ) {  // odd processes send, then receive
            MPI_Send(sendString, strlen(sendString)+1,
                       MPI_CHAR, id-1, 1, MPI_COMM_WORLD);
            MPI_Recv(receivedString, SIZE, MPI_CHAR, id-1, 2,
                       MPI_COMM_WORLD, &status);
        } else {          // even processes receive, then send
            MPI_Recv(receivedString, SIZE, MPI_CHAR, id+1, 1,
                       MPI_COMM_WORLD, &status);
            MPI_Send(sendString, strlen(sendString)+1,
                       MPI_CHAR, id+1, 2, MPI_COMM_WORLD);
        }

        printf("\nProcess %d of %d received the message:\n\t'%s'\n",
                id, numProcesses, receivedString);

        free(sendString);
        free(receivedString);
    } else if ( !id) {  // only process 0 does this part
        printf("\nPlease run this program using -np N where N is positive and even.\n\n");
    }

    MPI_Finalize();
    return 0;
}


In [ ]:
!mpicc -Wall -ansi -pedantic -std=c99 -o messagePassing2 messagePassing2.c 
!mpirun --allow-run-as-root -np 4 ./messagePassing2

### Exercise

- Run, using N = 4, 6, 8, and 10 processes. 
- In the above code, locate where the array of elements to be sent is being made by each process. What is different about each array per process?


## Ring of passed messages
Another pattern that appears in message passing programs is to use a ring of processes, where messages get sent in this fashion:

![picture of ring of message passing](images/ring.png)

When we have 4 processes, the idea is that process 0 will send data to process 1, who will receive it from process 0 and then send it to process 2, who will receive it from process 1 and then send it to process 3, who will receive it from process 2 and then send it back around to process 0.

In [ ]:
%%writefile messagePassing3.c
/* messagePassing3.c
 * ... illustrates the use of MPI_Send() and MPI_Recv(),
 *      in combination with the master-worker pattern.
 *
 * Joel Adams, Calvin College, November 2009.
 *
 * Usage: mpirun -np N ./messagePassing3
 *
 * Exercise:
 * - Run the program, varying the value of N from 1-8.
 * - Explain the behavior you observe.
 */

#include <stdio.h>    // printf()
#include <string.h>   // strlen()
#include <mpi.h>      // MPI

#define MAX 256

int main(int argc, char** argv) {
    int id = -1, numProcesses = -1;
    char sendBuffer[MAX] = {'\0'};
    char recvBuffer[MAX] = {'\0'};
    MPI_Status status;

    MPI_Init(&argc, &argv);
    MPI_Comm_rank(MPI_COMM_WORLD, &id);
    MPI_Comm_size(MPI_COMM_WORLD, &numProcesses);

    if (numProcesses > 1) {
        if ( id == 0 ) {                              // master:
            sprintf(sendBuffer, "%d", id);            //  create msg

            MPI_Send(sendBuffer,                      //  msg sent
                      strlen(sendBuffer) + 1,         //  num chars + NULL
                      MPI_CHAR,                       //  type
                      id+1,                           //  destination
                      1,                              //  tag
                      MPI_COMM_WORLD);                //  communicator

            MPI_Recv(recvBuffer,                      //  msg received
                      MAX,                            //  buffer size
                      MPI_CHAR,                       //  type
                      numProcesses-1,                 //  sender
                      1,                              //  tag
                      MPI_COMM_WORLD,                 //  communicator
                      &status);                       //  recv status

            printf("Process #%d of %d received %s\n", // show msg
                    id, numProcesses, recvBuffer);
        } else {                                      // workers:
            MPI_Recv(recvBuffer,                      //  msg received
                      MAX,                            //  buffer size
                      MPI_CHAR,                       //  type
                      MPI_ANY_SOURCE,                 //  sender (anyone)
                      1,                              //  tag
                      MPI_COMM_WORLD,                 //  communicator
                      &status);                       //  recv status

            printf("Process #%d of %d received %s\n", // show msg
                    id, numProcesses, recvBuffer);

            // build msg to send by appending id to msg received
            sprintf(sendBuffer, "%s %d", recvBuffer, id);

            MPI_Send(sendBuffer,                      //  msg to send
                      strlen(sendBuffer) + 1,         //  num chars + NULL
                      MPI_CHAR,                       //  type
                      (id+1) % numProcesses,          //  destination
                      1,                              //  tag
                      MPI_COMM_WORLD);                //  communicator
        }
    } else {
        printf("\nPlease run this program with at least 2 processes\n\n");
    }

    MPI_Finalize();
    return 0;
}


In [ ]:
!mpicc -Wall -ansi -pedantic -std=c99 -o messagePassing3 messagePassing3.c 
!mpirun --allow-run-as-root -np 4 ./messagePassing3

### Exercises
- Run, using N = from 1 through 8 processes.
- Make sure that you can trace how the code generates the output that you see.
- How is the finishing of the 'ring' completed, where the last process determines that it should send back to process 0?

# Collective Communication: Broadcast pattern
There are many cases when a master process obtains or creates data that needs to be sent to all of the other processes. There is a special pattern for this called **broadcast**. You will see examples of the master sending different types of data to each of the other processes.

## Broadcast from master to workers

We will look at three types of data that can be created in the master and sent to the workers. Rather than use send and receive, we will use a special new function called bcast.

![Important symbol](images/Important.jpg) **Note:** In each code example, note how the master does one thing, and the workers do another, but **all of the processes execute the bcast function.**


### Broadcast an integer

Find the place in this code where the data is being broadcast to all of the processes. Match the prints to the output you observe when you run it.

In [8]:
%%writefile broadcast.c
/* broadcast.c
 * ... illustrates the use of MPI_Bcast() with a scalar value...
 *      (compare to array version).
 * Joel Adams, Calvin College, April 2016.
 *
 * Usage: mpirun -np N ./broadcast
 *
 * Exercise:
 * - Compile and run several times,
 *     using 2, 4, and 8 processes
 * - Use source code to trace execution and output
 *     (noting contents of file "data.txt");
 * - Explain behavior/effect of MPI_Bcast().
 */

#include <mpi.h>
#include <stdio.h>
#include <stdlib.h>
#include <assert.h>

int main(int argc, char** argv) {
	int answer = 0;
	int numProcs = 0, myRank = 0;

	MPI_Init(&argc, &argv);
	MPI_Comm_size(MPI_COMM_WORLD, &numProcs);
	MPI_Comm_rank(MPI_COMM_WORLD, &myRank);

	if (myRank == 0) {
		answer = 42;
	}

	printf("BEFORE the broadcast, process %d's answer = %d\n",
	myRank, answer);

	MPI_Bcast(&answer, 1, MPI_INT, 0, MPI_COMM_WORLD);

	printf("AFTER the broadcast, process %d's answer = %d\n",
	myRank, answer);

	MPI_Finalize();

	return 0;
}


Writing broadcast.c


In [10]:
!mpicc -Wall -ansi -pedantic -std=c99 -o broadcast broadcast.c 
!mpirun --allow-run-as-root -np 4 ./broadcast

BEFORE the broadcast, process 1's answer = 0
BEFORE the broadcast, process 0's answer = 42
AFTER the broadcast, process 0's answer = 42
AFTER the broadcast, process 1's answer = 42


#### Exercise
- Run, using N = from 1 through 8 processes.

### Broadcast user input

The following program will take extra input that will get broadcast to all processes.

In [5]:
%%writefile broadcastUserInput.c
/* broadcastUserInput.c
 * ... illustrates the use of MPI_Bcast() with a scalar value
 *     obtained via a command line argument.
 *
 * Hannah Sonsalla, Macalester College 2017
 * Modeled from code by Joel Adams, Calvin College, April 2016.
 *
 * Usage: mpirun -np N ./broadcastUserInput <integer>
 *
 * Exercise:
 * - Compile and run several times varying the number
 *   of processes and integer value
 * - Explain the behavior you observe
 */

#include <mpi.h>
#include <stdio.h>
#include <stdlib.h>
#include <assert.h>

#define MASTER 0

/* gets value of answer from user
 * @param: argc, argument count.
 * @param: argv, argument pointer array.
 * @param: myRank, rank of current process
 * @param: answer, variable to store value given by user
 * Precondition: argc is a count of the number of arguments.
 *              && argv is a pointer array that points to the arguments.
 *              && myRank is the rank of this MPI process.
 *		&& answer is the variable to be assigned value.
 * Postcondition: answer has been filled with value from user
 *                if given, else answer remains set to 0.
 */
void getInput(int argc, char* argv[], int myRank, int* answer) {

    if (myRank == 0){  // master process
        if (argc == 2){
             *answer = atoi(argv[1]);
        }
    }
    MPI_Bcast(answer, 1, MPI_INT, 0, MPI_COMM_WORLD);
}

int main(int argc, char** argv) {
    int answer = 0, length = 0;
    int myRank = 0;

    char myHostName[MPI_MAX_PROCESSOR_NAME];

    MPI_Init(&argc, &argv);
    MPI_Comm_rank(MPI_COMM_WORLD, &myRank);
    MPI_Get_processor_name (myHostName, &length);

    printf("BEFORE the broadcast, process %d on host '%s' has answer = %d\n",
             myRank, myHostName, answer);

    getInput(argc, argv, myRank, &answer);

    printf("AFTER the broadcast, process %d on host '%s' has answer = %d\n",
             myRank, myHostName, answer);

    MPI_Finalize();

    return 0;
}


Writing broadcastUserInput.c


![warning sign](images/warning.png)
**Warning** This program is unlike any of the others and takes in a second argument, as shown below. 

In [7]:
!mpicc -Wall -ansi -pedantic -std=c99 -o broadcastUserInput broadcastUserInput.c 
!mpirun --allow-run-as-root -np 4 ./broadcastUserInput 42

BEFORE the broadcast, process 1 on host 'ip-172-31-58-111' has answer = 0
BEFORE the broadcast, process 0 on host 'ip-172-31-58-111' has answer = 0
AFTER the broadcast, process 0 on host 'ip-172-31-58-111' has answer = 42
AFTER the broadcast, process 1 on host 'ip-172-31-58-111' has answer = 42


#### Exercise
- Run, using N = from 1 through 8 processes, with an integer of your choosing.

### Broadcast an array

This is just one more example to show that other data structures can also be broadcast from the master to all worker processes.

In [ ]:
%%writefile broadcastSendReceive.c
/*
 * broadcastSendReceive.c
 * ... illustrates basic send receive functions.
 * Master process sends filled array to each process.
 *
 * Hannah Sonsalla, Macalester College 2017
 * fill and print function from code by Joel Adams, Calvin College
 *
 * Usage: mpirun -np N ./broadcastSendReceive
 *
 * Exercise:
 * - Compile and run, using 2, 4, and 8 processes
 * - Use source code to trace execution and output
 * 
 */

#include <mpi.h>
#include <stdio.h>
#include <stdlib.h>

/* fill an array with some arbitrary values 
 * @param: a, an int*.
 * @param: size, an int.
 * Precondition: a is the address of an array of ints.
 *              && size is the number of ints a can hold.
 * Postcondition: a has been filled with arbitrary values 
 *                { 11, 12, 13, ... }.
 */
void fill(int* a, int size) {
	int i;
	for (i = 0; i < size; i++) {
		a[i] = i+11;
	}
}

/* display a string, a process id, and its array values 
 * @param: str, a char*
 * @param: id, an int
 * @param: a, an int*.
 * Precondition: str points to either "BEFORE" or "AFTER"
 *              && id is the rank of this MPI process
 *              && a is the address of an 8-element int array.
 * Postcondition: str, id, and a have all been written to stdout.
 */
void print(char* str, int id, int* a) {
	printf("%s array sent, process %d has: {%d, %d, %d, %d, %d, %d, %d, %d}\n",
	   str, id, a[0], a[1], a[2], a[3], a[4], a[5], a[6], a[7]);
}

#define MAX 8

int main(int argc, char** argv) {
	int id = -1, numProcesses = -1;
	int array[MAX] = {0};
    

	MPI_Init(&argc, &argv);
	MPI_Comm_rank(MPI_COMM_WORLD, &id);
    	MPI_Comm_size(MPI_COMM_WORLD, &numProcesses);
    
	if (id == 0) fill(array, MAX);
     
	print("BEFORE", id, array);
	
	// master process sends array to every process
	if (id == 0) {
		for (int i = 1; i < numProcesses; i++) {
			MPI_Send(&array, MAX, MPI_INT, 
			    i, 1, MPI_COMM_WORLD);
	    }
	}
	
	else {
	    MPI_Recv(&array, MAX, MPI_INT, 0, 
	        1, MPI_COMM_WORLD, MPI_STATUS_IGNORE);
	}
	
    	print("AFTER", id, array);
 	MPI_Finalize();

	return 0;
}


In [ ]:
!mpicc -Wall -ansi -pedantic -std=c99 -o broadcastSendReceive broadcastSendReceive.c 
!mpirun --allow-run-as-root -np 4 ./broadcastSendReceive

#### Exercise
- Run, using N = from 1 through 8 processes.


# Collective Communication: reduction pattern

There are often cases when every process needs to complete a partial result of an overall computation. For example if you want to process a large set of numbers by summing them together into one value (i.e. *reduce* a set of numbers into one value, its sum), you could do this faster by having each process compute a partial sum, then have all the processes communicate to add each of their partial sums together.

This is so common in parallel processing that there is a special collective communication function called **reduce** that does just this.

## Collective Communication: reduce function

The type of reduction of many values down to one can be done with different types of operators on the set of values computed by each process.


### Reduce all values using sum and max
In this example, every process computes the square of (id+1). Then all those values are summed together and also the maximum function is applied.

In [ ]:
%%writefile reduction.c
/* reduction.c
* ... illustrates the use of MPI_Reduce()...
* Joel Adams, Calvin College, November 2009.
*
* Usage: mpirun -np N ./reduction
*
* Exercise:
* - Compile and run, varying N: 4, 6, 8, 10.
* - Explain behavior of MPI_Reduce().
*/

#include <mpi.h>
#include <stdio.h>
#include <stdlib.h>

int main(int argc, char** argv) {
    int numProcs = -1, myRank = -1, square = -1, max = -1, sum = 0;

    MPI_Init(&argc, &argv);
    MPI_Comm_size(MPI_COMM_WORLD, &numProcs);
    MPI_Comm_rank(MPI_COMM_WORLD, &myRank);

    square = (myRank+1) * (myRank+1);

    printf("Process %d computed %d\n", myRank, square);

    MPI_Reduce(&square, &sum, 1, MPI_INT, MPI_SUM, 0, MPI_COMM_WORLD);

    MPI_Reduce(&square, &max, 1, MPI_INT, MPI_MAX, 0, MPI_COMM_WORLD);

    if (myRank == 0) {
        printf("\nThe sum of the squares is %d\n\n", sum);
        printf("The max of the squares is %d\n\n", max);
    }

    MPI_Finalize();

    return 0;
}


In [ ]:
!mpicc -Wall -ansi -pedantic -std=c99 -o reduction reduction.c 
!mpirun --allow-run-as-root -np 4 ./reduction

#### Exercises
- Run, using N = from 1 through 8 processes.
- Try replacing MPI.MAX with MPI.MIN(minimum) and/or replacing MPI.SUM with MPI.PROD (product). Then save and run the code again.


### Reduction on an array of values

Here we will use reduction with arrays of values. Then note how you can change the semantics in the exercises.


In [ ]:
%%writefile reduction2.c
/* reduction2.c
 * ... illustrates the use of MPI_Reduce() using arrays...
 * Joel Adams, Calvin College, January 2015.
 *
 * Usage: mpirun -np 4 ./reduction2
 *
 * Exercise:
 * - Compile and run, comparing output to source code.
 * - Explain behavior of MPI_Reduce() in terms of
 *     srcArr and destArr.
 */

#include <mpi.h>
#include <stdio.h>

#define ARRAY_SIZE 5

void printArray(int id, char* arrayName, int* array, int SIZE);
void printSeparator(char* separator, int id);

int main(int argc, char** argv) {
    int myRank = -1;
    int srcArr[ARRAY_SIZE] = {0};
    int destArr[ARRAY_SIZE] = {0};

    MPI_Init(&argc, &argv);
    MPI_Comm_rank(MPI_COMM_WORLD, &myRank);

    if (myRank == 0) {
        printf("\nBefore reduction: ");
        printArray(myRank, "destArr", destArr, ARRAY_SIZE);
    }

    for (unsigned i = 0; i < ARRAY_SIZE; i++) {
        srcArr[i] = myRank * i;
    }

    printSeparator("", myRank);
    printArray(myRank, "srcArr", srcArr, ARRAY_SIZE);
    printSeparator("----", myRank);

    MPI_Reduce(srcArr, destArr, ARRAY_SIZE, MPI_INT, MPI_SUM, 0, MPI_COMM_WORLD);

    if (myRank == 0) {
        printf("\nAfter reduction:  ");
        printArray(myRank, "destArr", destArr, ARRAY_SIZE);
        printf("\n");
    }

    MPI_Finalize();

    return 0;
}

/* utility to display an array
 * params: id, the rank of the current process
 *         arrayName, the name of the array being displayed
 *         array, the array being displayed
 *         SIZE, the number of items in array.
 * postcondition:
 *         the id, name, and items in array have been printed to stdout.
 */
void printArray(int id, char* arrayName, int * array, int SIZE) {
    printf("Process %d, %s: [", id, arrayName);
    for (int i = 0; i < SIZE; i++) {
        printf("%3d", array[i]);
        if (i < SIZE-1) printf(",");
    }
    printf("]\n");
}

/* utility to print a separator string between before and after sections.
 * params: separator, a string
 *         id, the rank of the current process.
 * postcondition: the master process has printed the separator.
 */
void printSeparator(char* separator, int id) {
    MPI_Barrier(MPI_COMM_WORLD);
    if (id == 0) { printf("%s", separator); }
    MPI_Barrier(MPI_COMM_WORLD);
}




In [ ]:
!mpicc -Wall -ansi -pedantic -std=c99 -o reduction2 reduction2.c 
!mpirun --allow-run-as-root -np 4 ./reduction2

#### Exercises
- Run, using N = from 1 through 4 processes.
- Observe the new results and explain the behavior of MPI.SUM and MPI.MAX on an array of values.
- Can you define your own custom reduction function? How would you do this?

# Collective Communication: scatter and gather pattern

There are often cases when each process can work on some portion of a larger data structure. This can be carried out by having the master process maintain the larger structure and send parts to each of the worker processes, keeping part of the structure on the master. Each process then works on their portion of the data, and then the master can get the completed portions back.

This is so common in message passing parallel processing that there are two special collective communication functions called **scatter** and **gather** that handle this.


## Collective Communication: scatter and gather arrays

When several processes need to work on portions of a data structure, such as a 1-d or 2-d array, at various points in a program, a way to do this is to have one node, usually the master, divide the data structure and send portions to each of the other processes, often keeping one portion for itself. Each process then works on that portion of the data, and then the master can get the completed portions back. This type of coordination is so common that MPI has special patterns for it called **scatter** and **gather**.


### Scatter Arrays
The following diagrams illustrate how scatter using an array works. The master generates values in the array and all processes participate in the scatter:

![scatter array diagram](images/scatter_list_java.png)

After the scatter is completed, each process has one of the smaller array to work on, like this:

![after scatter array diagram](images/after_scatter_list_java.png)

In this next code example, an array is generated whose length is as twice the number of processes.

![Important symbol](images/Important.jpg) **Note:** In the code below, note how all processes must call the scatter function.

In [ ]:
%%writefile scatter.c
/* scatter.c
 * ... illustrates the use of MPI_Scatter()...
 * Joel Adams, Calvin College, November 2009.
 *
 * Usage: mpirun -np N ./scatter
 *
 * Exercise:
 * - Compile and run, varying N: 1, 2, 4, 8
 * - Trace execution through source code.
 * - Explain behavior/effect of MPI_Scatter().
 */

#include <mpi.h>      // MPI
#include <stdio.h>    // printf(), etc.
#include <stdlib.h>   // malloc()

void print(int id, char* arrName, int* arr, int arrSize);

int main(int argc, char** argv) {
    const int MAX = 8;
    int* arrSend = NULL;
    int* arrRcv = NULL;
    int numProcs = -1, myRank = -1, numSent = -1;

    MPI_Init(&argc, &argv);                            // initialize
    MPI_Comm_size(MPI_COMM_WORLD, &numProcs);
    MPI_Comm_rank(MPI_COMM_WORLD, &myRank);

    if (myRank == 0) {                                 // master process:
        arrSend = (int*) malloc( MAX * sizeof(int) );  //  allocate array1
        for (int i = 0; i < MAX; i++) {                //  load with values
            arrSend[i] = (i+1) * 11;
        }
        print(myRank, "arrSend", arrSend, MAX);        //  display array1
    }
     
    numSent = MAX / numProcs;                          // all processes:
    arrRcv = (int*) malloc( numSent * sizeof(int) );   //  allocate array2

    MPI_Scatter(arrSend, numSent, MPI_INT, arrRcv,     //  scatter array1 
                 numSent, MPI_INT, 0, MPI_COMM_WORLD); //   into array2

    print(myRank, "arrRcv", arrRcv, numSent);          // display array2

    free(arrSend);                                     // clean up
    free(arrRcv);
    MPI_Finalize();
    return 0;
}

void print(int id, char* arrName, int* arr, int arrSize) {
    printf("Process %d, %s: ", id, arrName);
    for (int i = 0; i < arrSize; i++) {
        printf(" %d", arr[i]);
    }
    printf("\n");
}



In [ ]:
!mpicc -Wall -ansi -pedantic -std=c99 -o scatter scatter.c 
!mpirun --allow-run-as-root -np 4 ./scatter

#### Exercises
- Run, using N = from 2 through 8 processes.
- If you want to study the code, explain to yourself how the array is generated by the master process and how it gets scattered to the worker processes.


### Gather Arrays
Once several processes have their own array of data, those array elements can also be gathered back together into a larger array, usually in the master process. All processes participate in a gather, like this:

![before gather diagram](images/gather_array_java.png)

The gather creates a combined array in the master, like this:

![after gather diagram](images/after_gather_array_java.png)

In this example, each process creates some very small arrays. Then a gather is used to create a combined array on the master process.

![Important symbol](images/Important.jpg) **Note:** In the code below, note how all processes must call the gather function.


In [ ]:
%%writefile gather.c
/* gather.c
 * ... illustrates the use of MPI_Gather()...
 * Joel Adams, Calvin College, November 2009.
 *
 * Usage: mpirun -np N ./gather
 *
 * Exercise:
 * - Compile and run, varying N: 1, 2, 4, 8.
 * - Trace execution through source.
 * - Explain behavior of MPI_Gather().
 */

#include <mpi.h>       // MPI
#include <stdio.h>     // printf()
#include <stdlib.h>    // malloc()

void print(int id, char* arrName, int* arr, int arrSize);

#define SIZE 3

int main(int argc, char** argv) {
   int  computeArray[SIZE];                          // array1
   int* gatherArray = NULL;                          // array2
   int  numProcs = -1, myRank = -1,
        totalGatheredVals = -1;

   MPI_Init(&argc, &argv);                           // initialize
   MPI_Comm_size(MPI_COMM_WORLD, &numProcs);
   MPI_Comm_rank(MPI_COMM_WORLD, &myRank);
                                                     // all processes:
   for (int i = 0; i < SIZE; i++) {                  //  load array1 with
      computeArray[i] = myRank * 10 + i;             //   3 distinct values
   }

   print(myRank, "computeArray", computeArray,       //  show array1
           SIZE);

   if (myRank == 0) {                                // master:
      totalGatheredVals = SIZE * numProcs;           //  allocate array2
      gatherArray = (int*) malloc( totalGatheredVals * sizeof(int) );
   }

   MPI_Gather(computeArray, SIZE, MPI_INT,           //  gather array1 vals
               gatherArray, SIZE, MPI_INT,           //   into array2
               0, MPI_COMM_WORLD);                   //   at master process

   if (myRank == 0) {                                // master process:
      print(myRank, "gatherArray",                   //  show array2
             gatherArray, totalGatheredVals);
      free(gatherArray);                             // clean up
   }


   MPI_Finalize();
   return 0;
}

void print(int id, char* arrName, int* arr, int arrSize) {
    printf("Process %d, %s: ", id, arrName);
    for (int i = 0; i < arrSize; i++) {
        printf(" %d", arr[i]);
    }
    printf("\n");
}


In [ ]:
!mpicc -Wall -ansi -pedantic -std=c99 -o gather gather.c 
!mpirun --allow-run-as-root -np 4 ./gather

#### Exercises
- Run, using N = from 2 through 8 processes.
- Try with different values of SIZE, perhaps changing printing of result for readability


## Collective Communication:  scatter and gather arrays

In this last scatter-gather example, a 1-D array is created by the master, then scattered, using scatter. After each smaller array used by each process is changed, the Gather (capital G) function brings the full array with the changes back into the master.

As before, the scatter function is used send portions of a larger array on the master to the workers, like this:

![alt text](images/Scatter_array.png)

The result of doing this then looks like this, where each process has a portion of the original that they can then work on:

![alt text](images/after_Scatter_array.png)

The reverse of this process is done using the gather function.

In this example,

![Important symbol](images/Important.jpg) **Note:** In the code below, note how all processes must call the scatter and gather functions.

In [ ]:
%%writefile scatterLoopGather.c
/* scatterLoopGather.c
 * ... scatters an array of data into equal-sized chunks, 
 *      has each process use a loop to double the values in its chunk,
 *      and then gathers the chunks back to the master process.
 *
 * Joel Adams, Calvin University, December 2019.
 *
 * Precondition: ARRAY_SIZE is evenly divisible by N
 *               && N <= ARRAY_SIZE.
 *
 * Note: The output of different process's steps will be interleaved
 *       (even using barriers) b/c stdout is buffered
 *       and MPI does not guarantee FIFO output behavior.
 *
 * Usage: mpirun -np N ./scatterLoopGather
 *
 * Exercise:
 * - Compile and run, using 1, 2, 4, and 8 processes
 * - Use source code to trace execution and output
 * - Explain behavior/effect of MPI_Scatter(), MPI_Gather().
 * - Optional: change ARRAY_SIZE to be another multiple of 8, such as 16
 * - Optional: add calls to print() to display each array at each step
 */

#include <stdio.h>     // printf
#include <stdlib.h>    // malloc, exit, ...
#include <mpi.h>       // MPI functionality

#define MASTER     0
#define ARRAY_SIZE 8

void fill(int* a, int size);
void printSeparator(const char* separator, int id);
void print(char* locLabel, int id, char* aName, int* a, int numElements);

/*
 *  Main function: double the values in an array
 *  by dividing the work equally among N processes.
 */
int main(int argc, char** argv) {
    int* scatterArray = NULL;
    int* chunkArray = NULL;
    int* gatherArray = NULL;
    int numProcs = -1, myRank = -1, chunkSize = -1;

    MPI_Init(&argc, &argv);
    MPI_Comm_size(MPI_COMM_WORLD, &numProcs);
    MPI_Comm_rank(MPI_COMM_WORLD, &myRank);

    printSeparator("", myRank);

    if (ARRAY_SIZE % numProcs || numProcs > ARRAY_SIZE) {
        if (myRank == MASTER) {
            printf("Please run with -np N divisible by and less than or equal to %d\n.", ARRAY_SIZE);
        }
        MPI_Finalize();
        exit(0);
    }

    if (myRank == MASTER) {     
        scatterArray = (int*) malloc( ARRAY_SIZE * sizeof(int) ); // allocate input array
        fill(scatterArray, ARRAY_SIZE);                           // populate it 
        gatherArray = (int*) malloc( ARRAY_SIZE * sizeof(int) );  // allocate result array
    }

    print("BEFORE Scatter", myRank, "scatterArray", scatterArray, ARRAY_SIZE);
    
    chunkSize = ARRAY_SIZE / numProcs;
    chunkArray = (int*) malloc(chunkSize * sizeof(int));          // allocate chunk array

    MPI_Scatter(scatterArray, chunkSize, MPI_INT,                 // scatter input array 
                 chunkArray, chunkSize, MPI_INT, 0, MPI_COMM_WORLD);

    print("AFTER Scatter", myRank, "chunkArray", chunkArray, chunkSize);

    for (unsigned i = 0; i < chunkSize; ++i) {                    // compute using chunk
        chunkArray[i] *= 2;
    }

    print("AFTER doubling", myRank, "chunkArray", chunkArray, chunkSize);

    MPI_Gather(chunkArray, chunkSize, MPI_INT,                   //  gather chunks
                gatherArray, chunkSize, MPI_INT,                 //   into gatherArray
                0, MPI_COMM_WORLD);

    print("AFTER gather", myRank, "gatherArray", gatherArray, ARRAY_SIZE);

    free(chunkArray);                                            // everyone clean up
    if (myRank == 0) {                                           // master clean up
        free(gatherArray); 
        free(scatterArray);
    }

    printSeparator("", myRank);

    MPI_Finalize();
    return 0;
}


/* fill an array with some easy-to-check values
 * @param: a, an int*.
 * @param: size, an int.
 * Precondition: a is the address of an array of ints.
 *              && size is the number of ints a can hold.
 * Postcondition: a has been filled with the values
 *                { 11, 12, 13, ... }.
 */
void fill(int* a, int size) {
    for (int i = 0; i < size; ++i) {
        a[i] = i+11;
    }
}

/* display a separator, synchronizing all processes
 * @param: separator, a char* 
 * @param: id, an int.
 * Precondition: separator points to a string to be be displayed
 *               && id is the MPI rank of this process.
 * Postcondition: separator has been displayed
 *                 and all MPI processes have been syncronized.
 */
void printSeparator(const char* separator, int id) {
    MPI_Barrier(MPI_COMM_WORLD);
    if (id == MASTER) { 
        printf("%s\n", separator);
    }
    MPI_Barrier(MPI_COMM_WORLD);
}

/* display a string, a process id, and its array values
 * @param: locLabel, a char*
 * @param: id, an int
 * @param: aName, a char*
 * @param: a, an int*.
 * @param: numElements, an int.
 * Precondition: locLabel points to a string describing our location
 *              && id is the rank of this MPI process
 *              && aName is the name of the array being printed
 *              && a is the address of an int array 
 *              && numElements is the number of int-values in a.
 * Postcondition: str, id, and a have all been written to stdout.
 */
void print(char* locLabel, int id, char* aName, int* a, int numElements) {
    printf("%s, process %d has this %s: {", locLabel, id, aName);
    if (a != NULL) {
        for (int i = 0; i < numElements - 1; ++i) {
            printf("%d, ", a[i]);
        }
        printf("%d}\n", a[numElements - 1]);
    } else {
        printf("}\n");
    }
}


In [ ]:
!mpicc -Wall -ansi -pedantic -std=c99 -o scatterLoopGather scatterLoopGather.c 
!mpirun --allow-run-as-root -np 4 ./scatterLoopGather

#### Exercises
- Run, using N = from 2 through 8 processes.
- What are the benefits and limitations of the scatter-gather functionality?


# When amount of work varies: balancing the load

There are algorithms where the master is used to assign tasks to workers by sending them data and receiving results back as each worker completes a task (or after the worker completes all of its tasks). In many of these cases, the computation time needed by each worker process for each of its tasks can vary somewhat dramatically. This situation is where **dynamic load balancing** can be helpful.

In this example we combine the master-worker pattern with message passing. The master has many tasks that need to be completed. The master starts by sending some data needed to complete a task to each worker process. Then the master loops and waits to hear back from each worker by receiving a message from any of them. When the master receives a message from a worker, it sends that worker more data for its next task, unless there are no more tasks to complete, in which case it sends a special message to the worker to stop running.

In this simple example, each worker is sent the number of seconds it should 'sleep', which can vary from 1 to 8. This illustrates varying sizes of workloads. Because of the code's simplicity, the number of tasks each worker does doesn't vary by much. In some real examples, the time for one task my be quite different than the time for another, which could have a different outcome, in which some workers were able to complete more tasks as others were doing long ones.

This approach can sometimes be an improvement on the assignment of an equal number of tasks to all processes.

Note in this case how the master, whose id is 0, handles the assignment of tasks, while the workers simply do what they are sent until they are told to stop.

In [ ]:
%%writefile dynamicLoadBalance.c
/* TODO: need to implement this */ 

In [ ]:
!mpicc -Wall -ansi -pedantic -std=c99 -o dynamicLoadBalance dynamicLoadBalance.c 
!mpirun --allow-run-as-root -np 4 ./dynamicLoadBalance

## Exercises
- Run, using N = 4 processes
- Study the execution carefully. Note that with 4 processes, 3 are workers. The total number of tasks is 3*4, or 12. Which process does the most work? You can count by looking for the lines that end with "... from X", where X is a worker process id.
- Try with N = 8 (7 workers).